In [1]:
import pandas as pd #biblioteca para manipulação de dados
import numpy as np #biblioteca para utilizacao de vetores e matrizes
import matplotlib.pyplot as plt #bibloteca para plotar graficos

In [2]:
#liberando acesso do colab aos arquivos no drive
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
#lendo o csv que contem a base de dados e armazanando em um df
df = pd.read_csv('/content/gdrive/My Drive/IGTI/Aulas praticas/wine.csv')

In [ ]:
df.head(10)

,class,Alcohol,Malic_acid,Ash,Alcalinity_of_ash,Magnesium,Total_phenols,Flavanoids,Nonflavanoid_phenols,Proanthocyanins,Color_intensity,Hue,OD280%2FOD315_of_diluted_wines,Proline
0,1,14.23,1.71,2.43,15.6,127,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065
1,1,13.20,1.78,2.14,11.2,100,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050
2,1,13.16,2.36,2.67,18.6,101,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185
3,1,14.37,1.95,2.50,16.8,113,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480
4,1,13.24,2.59,2.87,21.0,118,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735
5,1,14.20,1.76,2.45,15.2,112,3.27,3.39,0.34,1.97,6.75,1.05,2.85,1450
6,1,14.39,1.87,2.45,14.6,96,2.50,2.52,0.30,1.98,5.25,1.02,3.58,1290
7,1,14.06,2.15,2.61,17.6,121,2.60,2.51,0.31,1.25,5.05,1.06,3.58,1295
8,1,14.83,1.64,2.17,14.0,97,2.80,2.98,0.29,1.98,5.20,1.08,2.85,1045
9,1,13.86,1.35,2.27,16.0,98,2.98,3.15,0.22,1.85,7.22,1.01,3.55,1045


In [ ]:
df.describe()

,class,Alcohol,Malic_acid,Ash,Alcalinity_of_ash,Magnesium,Total_phenols,Flavanoids,Nonflavanoid_phenols,Proanthocyanins,Color_intensity,Hue,OD280%2FOD315_of_diluted_wines,Proline
count,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000
mean,1.938202,13.000618,2.336348,2.366517,19.494944,99.741573,2.295112,2.029270,0.361854,1.590899,5.058090,0.957449,2.611685,746.893258
std,0.775035,0.811827,1.117146,0.274344,3.339564,14.282484,0.625851,0.998859,0.124453,0.572359,2.318286,0.228572,0.709990,314.907474
min,1.000000,11.030000,0.740000,1.360000,10.600000,70.000000,0.980000,0.340000,0.130000,0.410000,1.280000,0.480000,1.270000,278.000000
25%,1.000000,12.362500,1.602500,2.210000,17.200000,88.000000,1.742500,1.205000,0.270000,1.250000,3.220000,0.782500,1.937500,500.500000
50%,2.000000,13.050000,1.865000,2.360000,19.500000,98.000000,2.355000,2.135000,0.340000,1.555000,4.690000,0.965000,2.780000,673.500000
75%,3.000000,13.677500,3.082500,2.557500,21.500000,107.000000,2.800000,2.875000,0.437500,1.950000,6.200000,1.120000,3.170000,985.000000
max,3.000000,14.830000,5.800000,3.230000,30.000000,162.000000,3.880000,5.080000,0.660000,3.580000,13.000000,1.710000,4.000000,1680.000000


In [ ]:
df.shape

(178, 14)

In [ ]:
df.columns

Index(['class', 'Alcohol', 'Malic_acid', 'Ash', 'Alcalinity_of_ash',
       'Magnesium', 'Total_phenols', 'Flavanoids', 'Nonflavanoid_phenols',
       'Proanthocyanins', 'Color_intensity', 'Hue',
       'OD280%2FOD315_of_diluted_wines', 'Proline'],
      dtype='object')

In [4]:
# armazenando os labels em um array
labels = np.array(df['class'])   # essa é a classe que a gente quer calcular a saída

# salvando a ordem das features
feature_list = list(df.columns)

In [5]:
df = df.drop('class', axis = 1)

In [ ]:
df.columns

Index(['Alcohol', 'Malic_acid', 'Ash', 'Alcalinity_of_ash', 'Magnesium',
       'Total_phenols', 'Flavanoids', 'Nonflavanoid_phenols',
       'Proanthocyanins', 'Color_intensity', 'Hue',
       'OD280%2FOD315_of_diluted_wines', 'Proline'],
      dtype='object')

In [6]:
data = np.array(df)

In [7]:
# faz o split dos dados
from sklearn.model_selection import train_test_split

train_data, test_data, train_labels, test_labels = train_test_split(data, labels, test_size = 0.25, random_state = 42)

In [8]:
# criando baseline
baseline_preds = np.random.choice([0,1,2], size = len(test_labels))

print(baseline_preds)

[1 1 1 2 0 2 0 1 1 0 2 0 2 2 2 1 0 1 2 2 0 2 1 1 2 0 2 0 1 1 2 2 0 2 0 0 1
 2 1 2 1 1 1 2 1]


In [9]:
# importar biblioteca para calculo de métricas
from sklearn import metrics  
from sklearn.metrics import cluster

# Avaliando o baseline!
# essas medidas são calculadas a partir da comparação com o valor real do nosso conjunto de teste
print('Coeficiente de Silhueta\n', metrics.silhouette_score(test_data, baseline_preds)) 
print('\nDavies-Bouldin Score\n', metrics.davies_bouldin_score(test_data, baseline_preds)) 

print('\nMatriz de Contingência\n', metrics.cluster.contingency_matrix(test_labels, baseline_preds)) 
print('\nMutual information\n', metrics.mutual_info_score(test_labels, baseline_preds)) 

Coeficiente de Silhueta
 -0.07703899127751966

Davies-Bouldin Score
 28.6403113620014

Matriz de Contingência
 [[3 4 8]
 [4 8 6]
 [4 5 3]]

Mutual information
 0.03163117766032014


In [10]:
# importar o modelo de KMeans
from sklearn.cluster import KMeans

In [11]:
import math

In [ ]:
# crio um método para auxiliar o cálculo ideal de clusters para o K-Means
def calculate_wcss(data):
    wcss = []
    for n in range(2, 7):
        kmeans = KMeans(n_clusters=n)
        kmeans.fit(X=data)
        wcss.append(kmeans.inertia_)

    return wcss

In [ ]:
# e aqui outro método com o cálculo ideal de clusters
def optimal_number_of_clusters(wcss):
    x1, y1 = 2, wcss[0]
    x2, y2 = 7, wcss[len(wcss)-1]

    distances = []
    for i in range(len(wcss)):
        x0 = i+2
        y0 = wcss[i]
        numerator = abs((y2-y1)*x0 - (x2-x1)*y0 + x2*y1 - y2*x1)
        denominator = math.sqrt((y2 - y1)**2 + (x2 - x1)**2)
        distances.append(numerator/denominator)
    
    return distances.index(max(distances)) + 2

In [ ]:
# calculando a soma dos quadrados para as 19 quantidade de clusters
sum_of_squares = calculate_wcss(df)

# calculando a quantidade ótima de clusters
n = optimal_number_of_clusters(sum_of_squares)


In [ ]:
n

4

In [13]:
# agora taca-lhe pau no k-means
clustering = KMeans(n_clusters = 3, random_state = 42)
 
# treinando o modelo no conjunto de dados de treino
clustering.fit(train_data);   # neste caso não precisa passar os labels de teste

In [14]:
predictions1_labels = clustering.predict(test_data)

In [15]:
#avaliando o modelo

print('Coeficiente de Silhueta\n', metrics.silhouette_score(test_data, predictions1_labels)) 
print('\nDavies-Bouldin Score\n', metrics.davies_bouldin_score(test_data, predictions1_labels)) 

print('\nMatriz de Contingência\n', metrics.cluster.contingency_matrix(test_labels, predictions1_labels)) 
print('\nMutual information\n', metrics.mutual_info_score(test_labels, predictions1_labels)) 

Coeficiente de Silhueta
 0.5519241838976592

Davies-Bouldin Score
 0.5415115100039725

Matriz de Contingência
 [[13  0  2]
 [ 0 12  6]
 [ 0  5  7]]

Mutual information
 0.526053317723497


In [16]:
pip install neptune-client

     |████████████████████████████████| 92kB 3.3MB/s 
     |████████████████████████████████| 829kB 7.2MB/s 
     |████████████████████████████████| 61kB 8.1MB/s 
     |████████████████████████████████| 204kB 19.1MB/s 
     |████████████████████████████████| 163kB 23.1MB/s 
     |████████████████████████████████| 71kB 8.2MB/s 
     |████████████████████████████████| 133kB 24.8MB/s 
     |████████████████████████████████| 71kB 8.8MB/s 
  Created wheel for neptune-client: filename=neptune_client-0.4.124-py2.py3-none-any.whl size=151386 sha256=4b8543fa94c9f60e5ccbe42026d98520e45ecbbdf8a38e7bad97aa3470006fe3
  Stored in directory: /root/.cache/pip/wheels/6a/e8/ff/67494e3d5076d2527b9b94e03c54866f9f99f1d9e1708f5fa7
  Created wheel for future: filename=future-0.18.2-cp36-none-any.whl size=491057 sha256=99281546d207b3fca5bff06ca2b180897d80b5bf0d051935334a0aeaa0a0ab46
  Stored in directory: /root/.cache/pip/wheels/8b/99/a0/81daf51dcd359a9377b110a8a886b3895921802d2fc1b2397e
  Created wheel for s

In [17]:
import neptune

NeptuneMissingApiTokenException: ignored